<a href="https://colab.research.google.com/github/manuelantelo/03MAIR-Algoritmos-de-Optimizacion/blob/master/Practica/Practica_Algoritmos_Manuel_Vicente_Antelo_Santos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Algoritmos de optimización - Seminario<br>
Nombre y Apellidos: **MANUEL VICENTE ANTELO SANTOS** <br>

Url Colab: https://colab.research.google.com/drive/1Ca_yxKr36Ge64O-DFINHha_yWp11Lfjg#scrollTo=hVbXYX-RfPWh

Url GitHbub: https://github.com/manuelantelo/03MAIR-Algoritmos-de-Optimizacion/blob/master/Practica/Practica_Algoritmos_Manuel_Vicente_Antelo_Santos.ipynb<br>

Problema:
> 1. Sesiones de doblaje <br>

Descripción del problema: Organizar sesiones de doblaje
* Se precisa coordinar el doblaje de una película. Los actores de doblaje deben coincidir en las tomas en las que sus personajes aparecen juntos en las diferentes tomas. Los actos de doblaje cobran todos la misma cantidad por cada día que deben desplazarse hasta el estudio de grabación independientemente del número de tomas que se graven. No es posible grabar más de 6 tomas por día. El objetivo es planificar las sesiones por día de manera que el gasto por los servicios de los actores de doblaje sea el menor posible.
  
* Los datos son:
  - Nº de actores = 10
  - Nº de tomas = 30
  - Matriz binaria de participación: A[i][j] = 1 si el actor j participa en la toma i.
  - Una toma puede grabarse un solo día.
  - Cada actor cobra una única vez por día, sin importar las tomas.
  - No se pueden grabar más de 6 tomas por día.
  -

* Objetivo: organizar las sesiones (días de grabación) para minimizar el coste total actor/día; es decir, minimizar el número de veces que los actores deben acudir al estudio ya que "los actores cobran lo mismo por cada día que se desplazan al estudio, indendientementel del nº de tomas que hagan".


**Modelado del problema**
Se representa la participación de los actores mediante una matriz binaria A[i][j]:
 - Filas: tomas (de 0 a 29).
 - Columnas: actores (de 0 a 9).

Técnica aplicada: **Programación entera binaria con restricciones (implementada mediante heurística voraz + mejora local por intercambio de variable)**
* Fase 1 (heurística voraz):
  - Se asignada cada toma al día que implique convocar al menor número de actores nuevos teniendo en cuenta que tiene un máximo de 6 tomas y se eligirá el día con menor coste.
    Esta decisión es localmente óptima y se toma en base a la diferencia de conjuntos: actores_nuevos = actores_toma - actores_ya_convocados_en_dia

* Fase 2 (mejora por intercambio):
  - Con la solución inicial se intenta intercambiar tomas entre pares de dias para ver si el intercambio de variable reduce el total de actores convocados y se aceptaría repitiéndose hasta no encontrar mejoras.


In [40]:
# Importar librerías
import numpy as np
import pandas as pd
import copy # para usar en copy.deepcopy() y crear una copia independiente de las listas y no modificar las originales

# Parámetros fijos (constantes del problema que se quiere solucionar)
NUM_ACTORES = 10
NUM_TOMAS = 30
MAX_DIAS = 6
MAX_TOMAS_POR_DIA = 6

# Matriz de participación
# Filas: tomas (0 a 29)
# Columnas: actores (0 a 9)
# A[i][j] = 1 si el actor j participa en la toma i
SESIONES = np.array([
    [1,1,1,1,1,0,0,0,0,0],  # Toma  0: Actores: 0, 1, 2, 3, 4
    [0,0,1,1,1,0,0,0,0,0],  # Toma  1: Actores: 2, 3, 4
    [0,1,0,0,1,0,1,0,0,0],  # Toma  2: Actores: 1, 4, 6
    [1,1,0,0,0,0,1,1,0,0],  # Toma  3: Actores: 0, 1, 6, 7
    [0,1,0,1,0,0,0,1,0,0],  # Toma  4: Actores: 1, 3, 7
    [1,1,0,1,1,0,0,0,0,0],  # Toma  5: Actores: 0, 1, 3, 4
    [1,1,0,1,1,0,0,0,0,0],  # Toma  6: Actores: 0, 1, 3, 4
    [1,1,0,0,0,1,0,0,0,0],  # Toma  7: Actores: 0, 1, 5
    [1,1,0,1,0,0,0,0,0,0],  # Toma  8: Actores: 0, 1, 3
    [1,1,0,0,0,1,0,0,1,0],  # Toma  9: Actores: 0, 1, 5, 8
    [1,1,1,0,1,0,0,1,0,0],  # Toma 10: Actores: 0, 1, 2, 4, 7
    [1,1,1,1,0,1,0,0,0,0],  # Toma 11: Actores: 0, 1, 2, 3, 5
    [1,0,0,1,1,0,0,0,0,0],  # Toma 12: Actores: 0, 3, 4
    [1,0,1,0,0,1,0,0,0,0],  # Toma 13: Actores: 0, 2, 5
    [1,1,0,0,0,0,1,0,0,0],  # Toma 14: Actores: 0, 1, 6
    [0,0,0,1,0,0,0,0,0,1],  # Toma 15: Actores: 3, 9
    [1,0,1,0,0,0,0,0,0,0],  # Toma 16: Actores: 0, 2
    [0,0,1,0,0,1,0,0,0,0],  # Toma 17: Actores: 2, 5
    [1,0,1,0,0,0,0,0,0,0],  # Toma 18: Actores: 0, 2
    [1,0,1,1,1,0,0,0,0,0],  # Toma 19: Actores: 0, 2, 3, 4
    [0,0,0,0,0,1,0,1,0,0],  # Toma 20: Actores: 5, 7
    [1,1,1,1,0,0,0,0,0,0],  # Toma 21: Actores: 0, 1, 2, 3
    [1,0,1,0,0,0,0,0,0,0],  # Toma 22: Actores: 0, 2
    [0,0,1,0,0,1,0,0,0,0],  # Toma 23: Actores: 2, 5
    [1,1,0,1,0,0,0,0,0,1],  # Toma 24: Actores: 0, 1, 3, 9
    [1,0,1,0,1,0,0,0,1,0],  # Toma 25: Actores: 0, 2, 4, 8
    [0,0,0,1,1,0,0,0,0,0],  # Toma 26: Actores: 3, 4
    [1,0,0,1,0,0,0,0,0,0],  # Toma 27: Actores: 0, 3
    [1,0,0,0,1,1,0,0,0,0],  # Toma 28: Actores: 0, 4, 5
    [1,0,0,1,0,0,0,0,0,0],  # Toma 29: actores: 0, 3
])

# -----------------------------------------------------------------------------------------------
# ANÁLISIS PREVIO DEL CONTENIDO DE LA MATRIZ SESIONES DE DOBLAJE
#
# Objetivo:
#   Obtener información estadística sobre la matriz de entrada 'SESIONES',
#   la cual indica qué actores participan y en qué tomas.
#
# Descripción de la matriz 'SESIONES':
#   - Matriz binaria de tamaño [n_tomas × n_actores].
#   - 'SESIONES[i][j] = 1' si el actor j participa en la toma i, y 0 en caso contrario.
#
# Información extraída:
#   - 'num_tomas': número total de tomas (filas de la matriz).
#   - 'num_actores': número total de actores (columnas de la matriz).
#   - 'sesiones_por_actor': cuantas tomas realiza cada actor (suma por columnas).
#   - 'actores_por_toma': cuantos actores hay en cada toma (suma por filas).
#   - 'actores_participantes': índices de actores que participan en al menos una toma.
#   - 'num_actores_que_participan': número total de actores activos (con >=1 participación).
#
# Esta información se usa para:
#   - Comprobar que la matriz está bien definida.
#   - Validar la participación global de los actores.
# -----------------------------------------------------------------------------------------------

# Nº de tomas
num_tomas = SESIONES.shape[0]
# Nº de actores
num_actores = SESIONES.shape[1]
# Nº total de actores
num_total_actores = SESIONES.shape[1]
# Número de sesiones por actor
sesiones_por_actor = SESIONES.sum(axis=0)
# Número de actores por toma
actores_por_toma = SESIONES.sum(axis=1)
# Actores únicos implicados en total
actores_participantes = np.nonzero(sesiones_por_actor)[0]
num_actores_que_participan = len(actores_participantes)

# Mostrar INFORME de la matriz 'SESIONES'
print(f"""
------------------------------------------------------
INFORME DEL CONTENIDO DE LA MATRIZ SESIONES DE DOBLAJE
------------------------------------------------------
- Número total de Tomas (filas): {num_tomas}
- Número total de Actores (columnas): {num_actores}
- Número de actores implicados en cada toma:
{dict((f"Toma {i + 1}", int(actores_por_toma[i])) for i in range(num_tomas))}
""")

# Crear DataFrame con la participación por actor
df_actor_tomas = pd.DataFrame({
    "Nº ACTOR": [i + 1 for i in range(len(sesiones_por_actor))],
    "Nº DE TOMAS PARTICIPA": sesiones_por_actor.astype(int)
})

print("\n[NUMERO DE TOMAS EN LAS QUE PARTICIPA CADA ACTOR]")
print(df_actor_tomas.to_string(index=False))

# Crear DataFrame con el resumen de participación por actores en las tomas
print("\n[RESUMEN DE PARTICIPACION DE LOS ACTORES]")
df_resumen_participacion = pd.DataFrame({
    "TOTAL_ACTORES": [num_total_actores],
    "ACTORES_QUE_PARTICIPAN_EN_ALGUNA_TOMA": [", ".join(map(str, actores_participantes + 1))],
    "Nº_ACTORES_PARTICIPANTES": [num_actores_que_participan],
    "CONCLUSION": [
        "Todos los actores participan en al menos una toma"
        if num_actores_que_participan == num_total_actores
        else "No todos los actores participan"
    ]
})

# --- Mostrar los resultados ---
print(df_resumen_participacion.to_string(index=False))


------------------------------------------------------
INFORME DEL CONTENIDO DE LA MATRIZ SESIONES DE DOBLAJE
------------------------------------------------------
- Número total de Tomas (filas): 30 
- Número total de Actores (columnas): 10
- Número de actores implicados en cada toma:
{'Toma 1': 5, 'Toma 2': 3, 'Toma 3': 3, 'Toma 4': 4, 'Toma 5': 3, 'Toma 6': 4, 'Toma 7': 4, 'Toma 8': 3, 'Toma 9': 3, 'Toma 10': 4, 'Toma 11': 5, 'Toma 12': 5, 'Toma 13': 3, 'Toma 14': 3, 'Toma 15': 3, 'Toma 16': 2, 'Toma 17': 2, 'Toma 18': 2, 'Toma 19': 2, 'Toma 20': 4, 'Toma 21': 2, 'Toma 22': 4, 'Toma 23': 2, 'Toma 24': 2, 'Toma 25': 4, 'Toma 26': 4, 'Toma 27': 2, 'Toma 28': 2, 'Toma 29': 3, 'Toma 30': 2}


[NUMERO DE TOMAS EN LAS QUE PARTICIPA CADA ACTOR]
 Nº ACTOR  Nº DE TOMAS PARTICIPA
        1                     22
        2                     14
        3                     13
        4                     15
        5                     11
        6                      8
        7       

In [41]:
# -----------------------------------------------------------------------------------------------
# HEURÍSTICA VORAZ INICIAL ADAPTATIVA
#
# Objetivo:
#   Construir una planificación inicial de las 30 tomas en un máximo de 6 días, MINIMIZANDO el coste en actor/día desde el principio.
#
# Método:
#   - Se recorre cada toma en orden.
#   - Para cada toma, se evalúa en qué día su asignación implicaría el menor número de actores nuevos (aún no convocados ese día).
#   - Se elige el día que minimiza ese coste adicional, siempre que no exceda el límite de 6 tomas por día.
#   - Una vez asignada una toma a un día, no se reubica (no hay retroceso).
#
# Características:
#   - Es una heurística voraz (greedy): toma decisiones localmente óptimas.
#   - Es adaptativa: tiene en cuenta qué actores ya están convocados.
#   - No va a garantizar la solución óptima pero produce una solución inicial
#     razonable (ver resultados fianales)
# -----------------------------------------------------------------------------------------------

# Creamos una lista vacía para cada día disponible para almacenar las tomas asignadas a ese día
dias = [[] for _ in range(MAX_DIAS)]

# Creamos una lista de listas donde almacenamos qué actores están ya convocados por día (el orden puede ser relevante y que no haya riesgo de duplicados)
actores_por_dia = [set() for _ in range(MAX_DIAS)]

# Recorremos cada toma de la película
for toma in range(NUM_TOMAS):
    # Crear el conjunto de actores que participan en la toma actual
    actores_toma = set(np.where(SESIONES[toma] == 1)[0])
    # Inicializamos el mejor día para asignar esta toma y el menor coste encontrado
    mejor_dia = -1
    menor_coste = float('inf')
    # Evaluamos todos los días posibles para asignar la toma
    for dia in range(MAX_DIAS):
        # Si el día ya tiene el máximo permitido de tomas, lo saltamos
        if len(dias[dia]) >= MAX_TOMAS_POR_DIA:
            continue
        # Calculamos qué actores nuevos son necesarios para esta toma (actores que participan en la toma actual MENOS actores ya convocados para ese día)
        actores_nuevos = actores_toma - actores_por_dia[dia]
        # El coste se define como el número de actores nuevos que habría que convocar
        coste = len(actores_nuevos)
        # Si este día tiene un coste menor que el menor encontrado hasta ahora, lo consideramos como el mejor
        if coste < menor_coste:
            menor_coste = coste
            mejor_dia = dia
    # Una vez encontrado el mejor día, añadimos la toma a ese día y actualizamos el conjunto de actores convocados ese día
    dias[mejor_dia].append(toma)
    actores_por_dia[mejor_dia].update(actores_toma)

In [32]:
# -----------------------------------------------------------------------------------------------
# FUNCION OBJETIVO para calcular el coste total en actor/día de una planificación dada.
# Para cada día, se cuenta una sola vez cada actor, sin importar cuántas tomas haga ese día (cobran lo mismo).
# El resultado es el número total de asistencias actor/día que se deben pagar.
#
# Parámetros de entrada:
#   - dias: lista de listas, donde cada sublista contiene los índices de las tomas asignadas a ese día.
#           Ejemplo: dias[0] = [0, 3, 5] -> día 1 tiene asignadas las tomas 1, 2, 3, 4 y 5.
#   - matriz: matriz binaria de tamaño [num_tomas x num_actores], donde matriz[i][j] = 1 si el actor j
#             participa en la toma i, y 0 en caso contrario.
#             Ejemplo: matriz[0] = [1, 1, 1, 1, 1, 0, ...] indica qué actores participan en la toma 1.
#
# Retorno:
#   - total: coste total de la planificación expresado como el número de asistencias actor/día;
#            es decir, cuántos "días de presencia" de actores se deben pagar en total.
#
# -----------------------------------------------------------------------------------------------
def calcular_coste_total(dias, matriz):
    total = 0  # Inicializamos el coste total
    # Recorremos cada día
    for dia in dias:
        actores = set()  # Conjunto para almacenar los actores únicos que participan ese día
        # Recorremos cada toma asignada a ese día
        for toma in dia:
            # Añadimos al conjunto los actores que participan en la toma t
            actores.update(np.where(matriz[toma] == 1)[0])
        # Sumamos al coste total el número de actores distintos que participan ese día
        total += len(actores)
    # Devolvemos el coste total (número de asistencias actor/día)
    return total

# Calculamos el coste de la solución mediante heurística voraz
coste_inicial = calcular_coste_total(dias, SESIONES)

In [50]:
# -----------------------------------------------------------------------------------------------
# BUSQUEDA LOCAL con estrategia de mejora por intercambio (Local Search)
# MEJORA POR INTERCAMBIO DE TOMAS ENTRE DÍAS-> intercambio de variable: [i], [j] = [j], [i]
#
# Objetivo:
#   Reducir el coste total de la planificación inicial obtenida por heurística voraz,
#   mediante pequeñas modificaciones locales: intercambio de tomas entre días.
#
# Método:
#   - Se recorre todos los pares de días posibles (i, j).
#   - Para cada par de días, se prueban todas las combinaciones posibles de intercambio
#     entre tomas del día i y tomas del día j.
#   - Cada vez que se prueba un intercambio:
#       • Se verifica que no se exceda el número máximo de tomas por día.
#       • Se calcula el nuevo coste de la planificación simulada.
#       • Si el nuevo coste es mejor, se acepta el intercambio.
#   - El proceso se repite mientras sigan encontrándose mejoras (búsqueda local iterativa).
#
# Características:
#   - No garantiza encontrar el óptimo global, pero mejora significativamente la solución inicial.
#   - Mantiene la factibilidad del problema (respeta restricciones).
#   - Creamos una copia de la planificación original para trabajar sobre ella sin modificar la original
#
# Resultado:
#   Se obtiene una planificación refinada (dias_mejorado) con menor coste actor/día.
# -----------------------------------------------------------------------------------------------

dias_mejorado = [list(d) for d in dias]
# Calculamos el coste inicial como punto de partida para la mejora
mejor_coste = calcular_coste_total(dias_mejorado, SESIONES)

# Variable de control para saber si se ha producido una mejora en una iteración
mejorado = True

# Repetir el proceso de mejora mientras sigamos encontrando soluciones mejores
while mejorado:
    mejorado = False  # Reiniciamos el estado de mejora en cada iteración
    # Recorremos todos los pares de días posibles (sin repetir)
    for i in range(len(dias_mejorado)):
        for j in range(i + 1, len(dias_mejorado)):
            # Probamos a intercambiar todas las combinaciones posibles de tomas entre los días i y j
            for toma_i in dias_mejorado[i]:
                for toma_j in dias_mejorado[j]:
                    # Creamos copias temporales de los dos días para simular el intercambio
                    temp_i = dias_mejorado[i].copy()
                    temp_j = dias_mejorado[j].copy()
                    # Realizamos el intercambio de tomas entre los dos días
                    temp_i.remove(toma_i)
                    temp_j.remove(toma_j)
                    temp_i.append(toma_j)
                    temp_j.append(toma_i)
                    # Verificamos que no se exceda el límite de tomas por día
                    if len(temp_i) > MAX_TOMAS_POR_DIA or len(temp_j) > MAX_TOMAS_POR_DIA:
                        continue  # Si no cumple, pasamos al siguiente par de variables
                    # Creamos una copia independiente (para no modificar la original) de toda la planificación para evaluar el nuevo coste
                    temp_dias = copy.deepcopy(dias_mejorado)
                    temp_dias[i] = temp_i
                    temp_dias[j] = temp_j
                    # Calculamos el nuevo coste total con el intercambio simulado
                    nuevo_coste = calcular_coste_total(temp_dias, SESIONES)
                    # Si el nuevo coste es mejor que el anterior, aceptamos el cambio
                    if nuevo_coste < mejor_coste:
                        dias_mejorado[i] = temp_i
                        dias_mejorado[j] = temp_j
                        mejor_coste = nuevo_coste
                        mejorado = True  # Indicamos que ha habido mejora para repetir el proceso
                        break  # Rompemos para aplicar la mejora
                if mejorado:
                    break
            if mejorado:
                break
        if mejorado:
            break


In [51]:
# -----------------------------------------------------------------------------------------------
# MEJORA MEDIANTE BÚSQUEDA TABÚ (tabu search)
#
# Objetivo:
#   Mejorar la solución obtenida por heurística voraz (incluso tras mejora local),
#   mediante la técnica metaheurística de Búsqueda Tabú.
#
# Descripción:
#   - Intercambios entre tomas de días distintos, permitiendo salir de óptimos locales.
#   - Se evita deshacer movimientos recientes usando una lista tabú.
#   - Se acepta un movimiento si mejora el coste, aunque esté en la lista tabú.
#   - La lista tabú es de longitud fija (FIFO) se van eliminando las soluciones más antiguas
#     suponiendo que las nuevas soluciones están ya suficientemente lejos en términos de vecindad
#     como para que sean consideradas nuevamente.
# -----------------------------------------------------------------------------------------------

dias_mejorado = [list(d) for d in dias]

# Calculamos el coste inicial como punto de partida para la mejora
mejor_coste = calcular_coste_total(dias_mejorado, SESIONES)
mejor_solucion = copy.deepcopy(dias_mejorado)
lista_tabu = []
LONGITUD_TABU = 30
MAX_ITERACIONES = 100

for iter in range(MAX_ITERACIONES):
    mejora = False
    mejor_movimiento = None
    mejor_nueva_solucion = None
    mejor_nuevo_coste = float('inf')

    for i in range(len(dias_mejorado)):
        for j in range(i + 1, len(dias_mejorado)):
            for toma_i in dias_mejorado[i]:
                for toma_j in dias_mejorado[j]:
                    movimiento = ((i, toma_i), (j, toma_j))
                    if movimiento in lista_tabu:
                        continue
                    # Creamos copias temporales de los dos días para simular el intercambio
                    temp_i = dias_mejorado[i].copy()
                    temp_j = dias_mejorado[j].copy()
                    # Realizamos el intercambio de tomas entre los dos días
                    temp_i.remove(toma_i)
                    temp_j.remove(toma_j)
                    temp_i.append(toma_j)
                    temp_j.append(toma_i)
                    # Verificamos que no se exceda el límite de tomas por día
                    if len(temp_i) > MAX_TOMAS_POR_DIA or len(temp_j) > MAX_TOMAS_POR_DIA:
                        continue   # Si no cumple, pasamos al siguiente par de variables
                    # Creamos una copia independiente (para no modificar la original) de toda la planificación para evaluar el nuevo coste
                    temp_dias = copy.deepcopy(dias_mejorado)
                    temp_dias[i] = temp_i
                    temp_dias[j] = temp_j
                    # Calculamos el nuevo coste total con el intercambio simulado
                    nuevo_coste = calcular_coste_total(temp_dias, SESIONES)
                    # Si el nuevo coste es mejor que el anterior, aceptamos el cambio
                    if nuevo_coste < mejor_nuevo_coste:
                        mejor_movimiento = movimiento
                        mejor_nueva_solucion = temp_dias
                        mejor_nuevo_coste = nuevo_coste
                        mejora = True  # Indicamos que ha habido mejora para repetir el proceso
    if mejora:
        dias_mejorado = mejor_nueva_solucion
        if mejor_nuevo_coste < mejor_coste:
            mejor_coste = mejor_nuevo_coste
            mejor_solucion = copy.deepcopy(dias_mejorado)
            coste_tabusearch = mejor_coste  # Guarda el coste final tras Tabu Search
        lista_tabu.append(mejor_movimiento)
        if len(lista_tabu) > LONGITUD_TABU:
            lista_tabu.pop(0)
    else:
        break

In [54]:
# -----------------------------------------------------------------------------------------------
# MOSTRAR UN ANÁLISIS DE RESULTADOS
#
# Objetivo:
#   Mostrar al usuario la asignación final de tomas por día junto con los actores convocados,
#   tanto para la heurística inicial como para la mejora local.
#   Evaluar el coste en actor/día y verificar cuántos días han sido realmente utilizados.
#
# Secciones:
#   1. Resultados de la heurística voraz inicial adaptativa:
#       - Se genera un resumen por día con tomas asignadas y actores convocados.
#       - Se calcula el coste total inicial en actor/día.
#
#   2. Resultados tras la mejora local por intercambio de tomas:
#       - Se genera un nuevo resumen actualizado por día.
#       - Se recalcula el coste total optimizado tras la mejora.
#
#   3. Conclusiones:
#       - Se compara el número de días utilizados frente a los disponibles.
#       - Se resumen las ventajas observadas (ahorro de días, reducción de coste).
#
# Nota:
#   Esta parte no modifica la solución, solo la representa de forma legible
#   y cuantifica su calidad mediante métricas relevantes (coste y uso de recursos).
# -----------------------------------------------------------------------------------------------
asignacion_inicial = []
for i, dia in enumerate(dias):
    actores = set()
    for toma in dia:
        actores.update(np.where(SESIONES[toma] == 1)[0])
    asignacion_inicial.append({
        "DIA": i + 1,
        "TOMAS ASIGNADAS": dia,
        "ACTORES CONVOCADOS": sorted(actores),
        "Nº ACTORES": len(actores)
    })

df_inicial = pd.DataFrame(asignacion_inicial)
print("\n[RESULTADOS HEURISTICA VORAZ ADAPTATIVA]")
print(df_inicial.to_string(index=False))
print(f"Coste total optimizado mediante Heurística voraz (asistencias actor/día): {coste_inicial}")

# Mostrar HEURÍSTICA MEJORADA por intercambio local
asignacion_final = []
for i, dia in enumerate(dias_mejorado):
    actores = set()
    for toma in dia:
        actores.update(np.where(SESIONES[toma] == 1)[0])
    asignacion_final.append({
        "DIA": i + 1,
        "TOMAS ASIGNADAS": dia,
        "ACTORES CONVOCADOS": sorted(actores),
        "Nº ACTORES": len(actores)
    })


# --- Resultados por Mejora Local Intercambio de Variable ---
df_final = pd.DataFrame(asignacion_final)
print("\n[RESULTADOS HEURISTICA MEJORA LOCAL POR INTERCAMBIO DE VARIABLE (tomas entre días)]")
print(df_final.to_string(index=False))

coste_mejora_local = mejor_coste  # Guarda el coste tras mejora local
print(f"Coste total optimizado mediante intercambio de variables (asistencias actor/día): {mejor_coste}")

# ---Conclusiones: verificar cuántos días se han utilizado
dias_utilizados_inicial = [i+1 for i, dia in enumerate(dias) if len(dia) > 0]
dias_utilizados_final = [i+1 for i, dia in enumerate(dias_mejorado) if len(dia) > 0]


# --- Resultados mediante Búsqueda Tabú ---
asignacion_tabusearch = []
for i, dia in enumerate(mejor_solucion):
    actores = set()
    for toma in dia:
        actores.update(np.where(SESIONES[toma] == 1)[0])
    asignacion_tabusearch.append({
        "DÍA": i + 1,
        "TOMAS ASIGNADAS": dia,
        "ACTORES CONVOCADOS": sorted(list(actores)),
        "Nº ACTORES" : len(actores)
    })

df_tabusearch = pd.DataFrame(asignacion_tabusearch)
print("\n[RESULTADOS FINALES CON BÚSQUEDA TABÚ]")
print(df_tabusearch.to_string(index=False))
print(f"Coste total optimizado mediante Búsqueda Tabú (asistencias actor/día): {mejor_coste}")

# --- CONCLUSIONES de todos los algoritmos utilizados ---
# Recalcular los costes de cada solución
# coste_inicial = calcular_coste_total(dias, SESIONES)
# coste_mejora_local = calcular_coste_total(dias_mejorado, SESIONES)
# coste_tabusearch = calcular_coste_total(mejor_solucion, SESIONES)

# Calcular los días utilizados por cada técnica
dias_utilizados_inicial = [i + 1 for i, dia in enumerate(dias) if len(dia) > 0]
dias_utilizados_final = [i + 1 for i, dia in enumerate(dias_mejorado) if len(dia) > 0]
dias_utilizados_tabusearch = [i + 1 for i, dia in enumerate(mejor_solucion) if len(dia) > 0]

# Construcción del DataFrame de resumen
# Evaluamos dinámicamente los comentarios en función de los resultados
comentario_mejora_local = (
    "Se han concentrado las tomas en menos días, reduciendo el coste."
    if coste_mejora_local < coste_inicial
    else "No se ha conseguido reducir el coste respecto a la heurística inicial."
)

comentario_tabusearch = (
    "Se ha alcanzado una solución con menor coste y uso eficiente de días mediante búsqueda tabú."
    if coste_tabusearch < coste_mejora_local
    else "La búsqueda tabú no ha mejorado el resultado de la mejora local."
)

# Construimos el DataFrame de conclusiones
df_conclusion = pd.DataFrame({
    "TECNICA": ["HEURISTICA VORAZ", "INTERCAMBIO VARIABLES", "BUSQUEDA TABÚ"],
    "DIAS utilizados": [len(dias_utilizados_inicial), len(dias_utilizados_final), len(dias_utilizados_tabusearch)],
    "DIAS disponibles": [MAX_DIAS] * 3,
    "DIAS libres": [
        MAX_DIAS - len(dias_utilizados_inicial),
        MAX_DIAS - len(dias_utilizados_final),
        MAX_DIAS - len(dias_utilizados_tabusearch)
    ],
    "COSTE (actor/día)": [coste_inicial, coste_mejora_local, coste_tabusearch],
    "COMENTARIO": [
        "Es posible planificar sin usar todos los días disponibles.",
        comentario_mejora_local,
        comentario_tabusearch
    ]
})

# Mostrar en consola
print("\n[CONCLUSIONES QUE SE OBTIENEN DESPUES DE EJECUTAR CADA ALGORITMO Y COMPARAR LOS RESULTADOS OBTENIDOS]")
print(df_conclusion.to_string(index=False))


[RESULTADOS HEURISTICA VORAZ ADAPTATIVA]
 DIA          TOMAS ASIGNADAS       ACTORES CONVOCADOS  Nº ACTORES
   1       [0, 1, 2, 3, 4, 5]    [0, 1, 2, 3, 4, 6, 7]           7
   2     [6, 7, 8, 9, 10, 11] [0, 1, 2, 3, 4, 5, 7, 8]           8
   3 [12, 13, 14, 15, 16, 17] [0, 1, 2, 3, 4, 5, 6, 9]           8
   4 [18, 19, 20, 21, 22, 23]    [0, 1, 2, 3, 4, 5, 7]           7
   5 [24, 25, 26, 27, 28, 29] [0, 1, 2, 3, 4, 5, 8, 9]           8
   6                       []                       []           0
Coste total optimizado mediante Heurística voraz (asistencias actor/día): 38

[RESULTADOS HEURISTICA MEJORA LOCAL POR INTERCAMBIO DE VARIABLE (tomas entre días)]
 DIA          TOMAS ASIGNADAS    ACTORES CONVOCADOS  Nº ACTORES
   1     [2, 3, 14, 0, 10, 4] [0, 1, 2, 3, 4, 6, 7]           7
   2 [23, 20, 13, 17, 28, 25]    [0, 2, 4, 5, 7, 8]           6
   3   [15, 24, 11, 21, 7, 9] [0, 1, 2, 3, 5, 8, 9]           7
   4  [19, 1, 26, 18, 16, 22]          [0, 2, 3, 4]           4
   5   

(*)¿Cuantas posibilidades hay sin tener en cuenta las restricciones?<br>
- Por cada una de las 30 tomas se tiene 6 opciones que son los días posibles.
  Entonces: $6^{30}$ = 221 mil billones de combinaciones posibles


¿Cuantas posibilidades hay teniendo en cuenta todas las restricciones.
- Las restribuciones son:
  
  a) Cada día puede contener como máximo 6 tomas y cada toma debe ser asignada a exactamente un día.

  b) total de días utilizados puede ser ≤ 6.

Buscamos calcular todas las formas posibles de repartir 30 tomas en 6 días con la condición de que cada tía tenga exactamente 6 tomas.

Entonces: $\frac{30!}{(6!)^6} \approx 1{,}896 \times 10^{15}$ posibilidades = **1.9 mil billones** de repartir las 30 tomas en 6 días cumpliendo las restricciones impuestas.
   
Conclusión:
Este problema no es viable usando un algoritmo de fuerza bruta.


Respuesta

Modelo para el espacio de soluciones<br>
(*) ¿Cual es la estructura de datos que mejor se adapta al problema? Argumentalo.(Es posible que hayas elegido una al principio y veas la necesidad de cambiar, arguentalo)


El espacio de soluciones representa el conjunto de todas las configuraciones posibles que cumplen las restricciones del problema.
Durante la primera fase debemos realizar un modelo del problema a resolver.

El espacio de soluciones lo forman todas las posibles asignaciones válidas, y la solución óptima es aquella que minimiza o maximiza una función objetivo bajo esas restricciones.

La solución es una forma de distribuir las 30 tomas en 6 días (con máximo 6 tomas por día), respetando:
 - Cada toma asignada una única vez.
 - Cada día con máximo 6 tomas.
 - Se busca minimizar el coste actor/día.

El espacio de soluciones es el conjunto de todas las particiones posibles del conjunto {0, 1, ..., 29} (30 tomas), en 6 subconjuntos disjuntos de tamaño ≤ 6, donde la unión de todos los subconjuntos cubra todas las tomas sin repeticiones.

Se define como: $$
S = \left\{ (D_1, D_2, \ldots, D_6) \mid D_i \subseteq \{0, \ldots, 29\},\ \bigcup D_i = \{0, \ldots, 29\},\ |D_i| \leq 6,\ D_i \cap D_j = \emptyset \text{ si } i \ne j \right\}
$$

La estructura más adecuada para representar la solución fue usar una "lista de listas" y no tuve necesidad de cambiar por los algoritmos que he usado.
Ventajas:
a) Cada sublista representa un día lo que facilita el cumplimiento de restricciones y la evaluación eficiente del coste actor/día así como contiene los índices de las tomas asignadas a ese día.
b) Es más fácil mover las tomas entre días, el intercambio de variables en las sublistas así como clonarlas.
c) Se calcula rápidamente los actores involucrados en cada día a partir de las tomas asignadas. Ejemplo np.where(SESIONES[toma] == 1)[0])

El manual de la asignatura sugiere que, para problemas de asignación, debemos construir representaciones estructuradas, modificables y evaluables rápidamente, especialmente si aplicamos técnicas heurísticas o metaheurísticas.



Según el modelo para el espacio de soluciones<br>
(*)¿Cual es la función objetivo?
La función objetivo consiste en ***minimizar el número total de actores/día**.

Dimensiones:
- (30 tomas x 10 actores)
- $A[t][a] = 1$ : el actor $a$ participa en la toma $t$  
- $A[t][a] = 0$ : el actor $a$ **no** participa en la toma $t$


Dado un vector de 6 días:

$$(D_1, D_2, \ldots, D_6)$$

y una matriz binaria $A \in \{0, 1\}^{30 \times 10}$, donde:

$$A[t][a] = 1 \quad \text{si el actor } a \text{ participa en la toma } t$$

La función objetivo $f$ se define como:

$$f(D_1, \ldots, D_6) = \sum_{i=1}^{6} \left| \bigcup_{t \in D_i} \text{actores}(t) \right|$$

Es decir, la suma del número de actores distintos convocados en cada día.

Para cada día $D_i$, se calcula el conjunto de actores distintos que participan en las tomas asignadas a ese día luego se suma el número de actores distintos convocados para ese día y como resultado se obtiene el **coste total del actor/día**

La **función objetivo** del modelo para el espacio de soluciones es: $$\min \sum_{i=1}^{6} \left| \bigcup_{t \in D_i} \text{actores}(t) \right|$$

(*)¿Es un problema de maximización o minimización?
Es un problema de minimización ya que es objetivo es minimizar el número total de actores convocados cada  día durante todas las sesiones de doblaje.
Ventajas:
- Llamar a menos actores por día va a reducir el coste de producción.
- Permite realizar más tomas con el mínimo número de actos disponibles por día.

Respuesta

Diseña un algoritmo para resolver el problema por fuerza bruta

In [36]:
# Fuerza Bruta para 6 tomas y 2 días
import numpy as np
import itertools

# Subconjunto de la matriz SESIONES (6 tomas x 10 actores)
SESIONES = np.array([
    [1,1,1,1,1,0,0,0,0,0],  # Toma  0: Actores: 0, 1, 2, 3, 4
    [0,0,1,1,1,0,0,0,0,0],  # Toma  1: Actores: 2, 3, 4
    [0,1,0,0,1,0,1,0,0,0],  # Toma  2: Actores: 1, 4, 6
    [1,1,0,0,0,0,1,1,0,0],  # Toma  3: Actores: 0, 1, 6, 7
    [0,1,0,1,0,0,0,1,0,0],  # Toma  4: Actores: 1, 3, 7
    [1,1,0,1,1,0,0,0,0,0],  # Toma  5: Actores: 0, 1, 3, 4
    [1,1,0,1,1,0,0,0,0,0],  # Toma  6: Actores: 0, 1, 3, 4
    [1,1,0,0,0,1,0,0,0,0],  # Toma  7: Actores: 0, 1, 5
    [1,1,0,1,0,0,0,0,0,0],  # Toma  8: Actores: 0, 1, 3
    [1,1,0,0,0,1,0,0,1,0],  # Toma  9: Actores: 0, 1, 5, 8
    [1,1,1,0,1,0,0,1,0,0],  # Toma 10: Actores: 0, 1, 2, 4, 7
    [1,1,1,1,0,1,0,0,0,0],  # Toma 11: Actores: 0, 1, 2, 3, 5
    [1,0,0,1,1,0,0,0,0,0],  # Toma 12: Actores: 0, 3, 4
    [1,0,1,0,0,1,0,0,0,0],  # Toma 13: Actores: 0, 2, 5
    [1,1,0,0,0,0,1,0,0,0],  # Toma 14: Actores: 0, 1, 6
    [0,0,0,1,0,0,0,0,0,1],  # Toma 15: Actores: 3, 9
    [1,0,1,0,0,0,0,0,0,0],  # Toma 16: Actores: 0, 2
    [0,0,1,0,0,1,0,0,0,0],  # Toma 17: Actores: 2, 5
    [1,0,1,0,0,0,0,0,0,0],  # Toma 18: Actores: 0, 2
    [1,0,1,1,1,0,0,0,0,0],  # Toma 19: Actores: 0, 2, 3, 4
    [0,0,0,0,0,1,0,1,0,0],  # Toma 20: Actores: 5, 7
    [1,1,1,1,0,0,0,0,0,0],  # Toma 21: Actores: 0, 1, 2, 3
    [1,0,1,0,0,0,0,0,0,0],  # Toma 22: Actores: 0, 2
    [0,0,1,0,0,1,0,0,0,0],  # Toma 23: Actores: 2, 5
    [1,1,0,1,0,0,0,0,0,1],  # Toma 24: Actores: 0, 1, 3, 9
    [1,0,1,0,1,0,0,0,1,0],  # Toma 25: Actores: 0, 2, 4, 8
    [0,0,0,1,1,0,0,0,0,0],  # Toma 26: Actores: 3, 4
    [1,0,0,1,0,0,0,0,0,0],  # Toma 27: Actores: 0, 3
    [1,0,0,0,1,1,0,0,0,0],  # Toma 28: Actores: 0, 4, 5
    [1,0,0,1,0,0,0,0,0,0],  # Toma 29: actores: 0, 3
])

SESIONES_REDUCIDA = SESIONES[:6] # Solo las 6 primeras porque es inviable aumentar

# Todas las particiones válidas de 6 tomas en 2 días de 3 tomas
tomas = list(range(6))
mejor_coste = float('inf')
mejor_asignacion = None

# Generar todas las combinaciones de 3 tomas para el primer día
for dia1 in itertools.combinations(tomas, 3):
    dia1 = set(dia1)
    dia2 = set(tomas) - dia1
    asignacion = [list(dia1), list(dia2)]

    # Calcular coste actor/día
    coste = 0
    for dia in asignacion:
        actores = set()
        for toma in dia:
            actores.update(np.where(SESIONES_REDUCIDA[toma] == 1)[0])
        coste += len(actores)

    if coste < mejor_coste:
        mejor_coste = coste
        mejor_asignacion = asignacion

# Mostrar resultado
print("[FUERZA BRUTA - PRUEBA CON 6 TOMAS]")
for i, dia in enumerate(mejor_asignacion):
    actores = set()
    for toma in dia:
        actores.update(np.where(SESIONES_REDUCIDA[toma] == 1)[0])
    print(f"Día {i+1}: Tomas {dia} Actores: {sorted(list(actores))} (Nº actores: {len(actores)})")
print(f"\nCoste total mínimo: {mejor_coste} actor/día")


[FUERZA BRUTA - PRUEBA CON 6 TOMAS]
Día 1: Tomas [0, 1, 5] Actores: [np.int64(0), np.int64(1), np.int64(2), np.int64(3), np.int64(4)] (Nº actores: 5)
Día 2: Tomas [2, 3, 4] Actores: [np.int64(0), np.int64(1), np.int64(3), np.int64(4), np.int64(6), np.int64(7)] (Nº actores: 6)

Coste total mínimo: 11 actor/día


Calcula la complejidad del algoritmo por fuerza bruta

Respuesta:

El algoritmo por fuerza bruta explora **todas las asignaciones posibles** de 30 tomas a 6 días, cumpliendo que:
- Cada día tenga exactamente 6 tomas.
- Cada toma se asigne una sola vez.
- Se minimice el coste total actor/día.

El número total de particiones válidas del conjunto $\{0, \ldots, 29\}$ en 6 subconjuntos disjuntos de tamaño 6 es:

$$
\frac{30!}{(6!)^6} \approx 1.896 \times 10^{15}
$$

Cada solución se evalúa en tiempo constante $\mathcal{O}(1)$ porque:
- Hay 6 días.
- Cada día tiene 6 tomas como máximo.
- Cada toma tiene como mucho 10 actores.

Por tanto, la la **complejidad total** del algoritmo es: $$\mathcal{O}\left( \frac{30!}{(6!)^6} \right)$$

Esto lo hace **computacionalmente inviable** para 30 tomas.  
Solo puede utilizarse con instancias reducidas (por ejemplo, 6 u 8 tomas) como he puesto en el código para validación.

(*)Diseña un algoritmo que mejore la complejidad del algoritmo por fuerza bruta. Argumenta porque crees que mejora el algoritmo por fuerza bruta
Propongo el algoritmo de [Ramificación y Poda] para reducir el el nº de soluciones exploradas ya que no explora todas las soluciones, parte del espacio de soluciones de fuerza bruta, usa cotas inferiores para evitar recorrer ramas no prometedoras y; en el peor caso, puede ser igual de lento, pero en la práctica es mucho más eficiente.


In [37]:
import pandas as pd

# Subconjunto reducido de la matriz SESIONES para 6 tomas y 10 actores
SESIONES_REDUCIDA = np.array([
    [1, 0, 1, 0, 1, 0, 0, 1, 0, 0],
    [0, 1, 0, 1, 0, 0, 1, 0, 0, 1],
    [1, 1, 0, 0, 0, 1, 1, 0, 0, 0],
    [0, 0, 1, 1, 0, 0, 0, 0, 1, 1],
    [1, 0, 0, 0, 1, 1, 0, 1, 0, 0],
    [0, 1, 1, 0, 0, 0, 1, 0, 1, 0]
])

MAX_TOMAS = 6
MAX_TOMAS_DIA = 3
MAX_DIAS = 2
NUM_ACTORES = 10

mejor_coste = float('inf')
mejor_solucion = None

def coste(solucion, sesiones):
    total = 0
    for dia in solucion:
        actores = set()
        for toma in dia:
            actores.update(np.where(sesiones[toma] == 1)[0])
        total += len(actores)
    return total

def ramificacion_y_poda(solucion_parcial, tomas_disponibles, sesiones):
    global mejor_coste, mejor_solucion
    if not tomas_disponibles:
        c = coste(solucion_parcial, sesiones)
        if c < mejor_coste:
            mejor_coste = c
            mejor_solucion = [list(d) for d in solucion_parcial]
        return

    toma = tomas_disponibles[0]
    for i in range(MAX_DIAS):
        if len(solucion_parcial[i]) < MAX_TOMAS_DIA:
            solucion_parcial[i].append(toma)

            cota = coste(solucion_parcial, sesiones)
            if cota < mejor_coste:
                ramificacion_y_poda(solucion_parcial, tomas_disponibles[1:], sesiones)

            solucion_parcial[i].pop()

# Ejecutar
sol_inicial = [[] for _ in range(MAX_DIAS)]
ramificacion_y_poda(sol_inicial, list(range(MAX_TOMAS)), SESIONES_REDUCIDA)

# Mostrar resultados
resultado = []
for i, dia in enumerate(mejor_solucion):
    actores = set()
    for toma in dia:
        actores.update(np.where(SESIONES_REDUCIDA[toma] == 1)[0])
    resultado.append({
        "DÍA": i + 1,
        "TOMAS ASIGNADAS": dia,
        "ACTORES CONVOCADOS": sorted(list(actores)),
        "Nº ACTORES": len(actores)
    })

df_rp = pd.DataFrame(resultado)
print("\n[RESULTADO RAMIFICACIÓN Y PODA (6 TOMAS)]")
print(df_rp.to_string(index=False))
print(f"\nCoste total mínimo: {mejor_coste} actores/día")



[RESULTADO RAMIFICACIÓN Y PODA (6 TOMAS)]
 DÍA TOMAS ASIGNADAS    ACTORES CONVOCADOS  Nº ACTORES
   1       [0, 2, 4] [0, 1, 2, 4, 5, 6, 7]           7
   2       [1, 3, 5]    [1, 2, 3, 6, 8, 9]           6

Coste total mínimo: 13 actores/día


(*) Calcula la complejidad del algoritmo

El algoritmo de Ramificación y Poda explora un árbol de decisiones en el que cada nivel representa la asignación de una toma a un día, y aplica poda para eliminar ramas no prometedoras.
Si no hubiera poda, cada una de las $n = 30$ tomas podría asignarse a uno de los 6 días:
$$
\text{Nº de nodos sin poda} \leq 6^{30}
$$

Esto es una complejidad exponencial inasumible.

Se podan ramas del árbol cuando:
- Se supera el límite de 6 tomas por día (es inviable a partir de ese número).
- La cota inferior del nodo actual es mayor o igual que la mejor solución conocida.

Esto reduce enormemente el número de nodos explorados en la práctica.

Complejidad:
**Peor caso (sin poda efectiva):** $$\mathcal{O}(6^n)\quad \text{donde } n = \text{nº de tomas}$$

**Mejor caso (con poda eficiente):**
  - Explora solo una pequeña fracción del árbol.
  - Solución óptima o casi óptima en tiempo razonable y más rápido que con fuerza bruta.

Aunque [Ramificación y Poda] tiene también complejidad exponencial en el peor caso, la utilización de poda basada en cotas permite resolver instancias que sería imposibles popr [Fuerza Bruta].

**Según el problema (y tenga sentido), diseña un juego de datos de entrada aleatorios**

In [38]:
def generar_matriz_sesiones(num_tomas, num_actores, min_actores, max_actores, seed):
    """
    Genera una matriz binaria SESIONES de tamaño (num_tomas x num_actores),
    donde cada fila representa una toma y cada columna representa un actor.

    Parámetros:
    - num_tomas: número total de tomas (filas de la matriz)
    - num_actores: número total de actores (columnas de la matriz)
    - min_actores: número mínimo de actores por toma
    - max_actores: número máximo de actores por toma
    - seed: semilla para datos aleatorios

    Retorna:
    - SESIONES: matriz binaria aleatoria de dimensiones (num_tomas x num_actores)
    """

    # Fijamos la semilla para obtener resultados
    np.random.seed(seed)

    # Inicializamos la matriz SESIONES con ceros (ningún actor participa aún)
    SESIONES = np.zeros((num_tomas, num_actores), dtype=int)

    # Para cada toma (fila), elegimos un número aleatorio de actores que participarán
    for toma in range(num_tomas):
        # Seleccionamos aleatoriamente entre min_actores y max_actores
        n_actores = np.random.randint(min_actores, max_actores + 1)

        # Elegimos 'n_actores' actores diferentes (sin repetición) de entre todos los disponibles
        actores_en_toma = np.random.choice(num_actores, size=n_actores, replace=False)

        # Asignamos un 1 en la matriz para indicar qué actores participan en esta toma
        SESIONES[toma, actores_en_toma] = 1

    return SESIONES

**Aplica el algoritmo al juego de datos generado**

In [39]:
# Generamos una matriz SESIONES aleatoria de 30 tomas y 10 actores
NUM_TOMAS = 30
NUM_ACTORES = 10
MIN_ACTORES = 2
MAX_ACTORES = 5
SEED = 42

SESIONES = generar_matriz_sesiones(
    num_tomas=NUM_TOMAS,
    num_actores=NUM_ACTORES,
    min_actores=MIN_ACTORES,
    max_actores=MAX_ACTORES,
    seed=SEED
)

# Mostrar las primeras filas de la matriz generada
print("Primeras 5 tomas de la matriz SESIONES:")
print(SESIONES[:5])

Primeras 5 tomas de la matriz SESIONES:
[[1 0 0 0 0 1 1 0 0 1]
 [1 1 0 0 0 0 1 0 1 1]
 [0 0 1 0 0 0 1 0 1 0]
 [0 1 0 0 1 1 0 0 1 0]
 [0 0 1 0 0 1 0 1 0 1]]


**Enumera las referencias que has utilizado(si ha sido necesario) para llevar a cabo el trabajo**

He tenido que mirar el material siguiente:
- Manual de la asignatura 'Algoritmos de Optimización' (para decidir qué algoritmos eran propios para resolver el problema).
- Manual de la asignatura 'Matemáticas para la IA' (para resolver lo referente a la combinatoria).
- Manual de la asignatura 'Python para la IA' (para resolver el código en python).
- Web de (numphy) https://numpy.org/ para maniputlación de matrices (modelo de datos de SESIONES, 'itertools' para las combinaciones por fuerza bruta y 'pandas' para visualizar las soluciones.

**Describe brevemente las lineas de cómo crees que es posible avanzar en el estudio del problema. Ten en cuenta incluso posibles variaciones del problema y/o variaciones al alza del tamaño**

El problema de organizar las sesiones de doblaje es un problema de asignación  con restricciones combinatorias ya comentadas anteriormente y su escalabilidad habería que analizarlo; por ejemplo, si se aumentase el número de tomas y de actores se podería evaluar el comportamiento de los algoritmos analizando cómo va creciendo la complejidad o se degrada la solución con las técnicas heurística y metaheurística.